[View in Colaboratory](https://colab.research.google.com/github/kimitakem/colab_ex/blob/master/sample.ipynb)

# Tensorflow: Eager executionモード
## 初期化：
- ライブラリのインポート
- Eagerモードのアクティベーションを行う


In [1]:
import tensorflow as tf
tf.enable_eager_execution()

## 環境の確認
- Tensorflowのバージョン
- Eagerモードになっているかの確認

In [2]:
print("Tensorflow Version: {}".format(tf.__version__))
print("Check Eager Mode: {}".format(tf.executing_eagerly()))
tfe = tf.contrib.eager


Tensorflow Version: 1.9.0
Check Eager Mode: True


## GradientTapeを使った微分演算
- 変数の定義：`tf.contrib.eager.Variable`を使う。
- with文の中でGradientTapeを開き、数式間の関係性を定義する
- `GradientTape.gradient`で微分を算出する


In [21]:
w_scalar = tfe.Variable(5.0)
w_matrix = tfe.Variable([[5.0, 4.0], [3.0, 2.0]])

with tf.GradientTape() as tape:
    loss_scalar = w_scalar * w_scalar

print("loss_scalarのw_scalarに対する微分")    
print(tape.gradient(loss_scalar, w_scalar))

with tf.GradientTape() as tape:
    loss_mat = w_matrix * w_matrix
    
print("loss_matrixのw_matrixに対する微分")    
print(tape.gradient(loss_mat, w_matrix))    


tf.Tensor(10.0, shape=(), dtype=float32)
tf.Tensor(
[[10.  8.]
 [ 6.  4.]], shape=(2, 2), dtype=float32)


## 単純モデルの学習
- 線形モデルの回帰


In [31]:
NUM_EXAMPLES = 1000
gt_weight = 4
gt_bias = 3
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * gt_weight + gt_bias + noise

print("Training Input:{}...".format(training_inputs[:5]))
print("Training Output:{}...".format(training_outputs[:5]))


def prediction(input, weight, bias):
    return input * weight + bias


# Loss Function
def loss(weights, biases):
    error = prediction(training_inputs, weights, biases) - training_outputs
    return tf.reduce_mean(tf.square(error))


def grad(weights, biases):
    with tf.GradientTape() as tape:
        loss_value = loss(weights, biases)
    return tape.gradient(loss_value, [weights, biases])


training_steps = 100
learning_rate = 0.04

W = tfe.Variable(5.)
B = tfe.Variable(10.)

print("Initial Loss: {:.3f}".format(loss(W, B)))

for i in range(training_steps):
    dW, dB = grad(W, B)
    W.assign_sub(dW * learning_rate)
    B.assign_sub(dB * learning_rate)
    if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(W, B)))

print("Final Loss: {:.3f}".format(loss(W, B)))
print("W = {} / W_gt = {}".format(W.numpy(), gt_weight))
print("B = {} / B_gt = {}".format(B.numpy(), gt_bias))



Training Input:[ 0.35780424 -0.2101717  -1.2546268  -0.5493252   0.04965408]...
Training Output:[ 4.6908164   3.7751508  -2.6632924   0.68101513  2.271222  ]...
Initial Loss: 50.576
Loss at step 000: 43.030
Loss at step 020: 2.507
Loss at step 040: 0.995
Loss at step 060: 0.938
Loss at step 080: 0.936
Final Loss: 0.936
W = 4.006326198577881 / W_gt = 4
B = 2.9826340675354004 / B_gt = 3


## Mnistの学習
#### 事前準備
- 実行前に、https://github.com/tensorflow/models.gitをcloneしpythonpathを通す
- pycharmのpythonpathの設定はproject interpretorの設定から

In [32]:
import official.mnist

In [34]:
from official.mnist import dataset as mnist_dataset
mnist_dataset.train("")

PermissionError: [WinError 32] プロセスはファイルにアクセスできません。別のプロセスが使用中です。: 'C:\\Users\\kimim\\AppData\\Local\\Temp\\tmp14f5s5x7.gz'

In [7]:
!cd


C:\Users\kimim\PycharmProjects\colab_ex
